In [1]:
import pandas as pd
import numpy as np

def convert_time(time):
    # return time in hours and correct for day overflow (datetime object has attribute year only if it is more than 24h after start)
    '''Converts the data format into seconds'''
    h,m,s = (time.hour,time.minute,time.second)
    return (int(h) * 3600 + int(m) * 60 + int(s))/3600 + (24 if hasattr(time,"year") else 0)


def get_dataframe_from_xlsx(path):
    df = pd.read_excel(path,header=None)

    df_list = np.split(df, df[df.isnull().all(1)].index)
    df_list_new = []
    for i in range(len(df_list)):
        df = df_list[i]
        df.columns = df.iloc[0 if i == 0 else 1] #set header
        df = df.iloc[1 if i == 0 else 2:] #delete first row (header)
        df = df.set_index('Kinetic read') #set index
        df.index = df.index.map(convert_time) #convert datetime.time into seconds
        df = df[df.columns.dropna()] #remove empty columns
        df = df.dropna() #remove all empty lines
        df_list_new.append(df)
    return pd.concat(df_list_new,axis=1)

df_msn4 = get_dataframe_from_xlsx('iGEM-EPFL-2020-Data_Analysis/data/202000814_iGEM2020-KO-Fitness-plate7.xlsx')
df_wt = get_dataframe_from_xlsx('iGEM-EPFL-2020-Data_Analysis/data/202000808_iGEM2020-KO-Fitness-plate5.xlsx')
df_wt = df_wt.loc[:,~df_wt.columns.duplicated()] # remove duplicate columns
df_msn2 = get_dataframe_from_xlsx('iGEM-EPFL-2020-Data_Analysis/data/202000809_iGEM2020-KO-Fitness-plate6.xlsx')
df_yap1 = get_dataframe_from_xlsx('iGEM-EPFL-2020-Data_Analysis/data/202000815_iGEM2020-KO-Fitness-plate8.xlsx')

In [2]:
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
import sys
from scipy.optimize import curve_fit
import string
import matplotlib.pyplot as plt

pesticides = ['Bentazone','Atrazine desisoprpyl','Chloridazone','DDT','Diazinon','Metolachlore','Simazine','Atrazine','Control']
alph = string.ascii_uppercase

def logistic(x,x0,y0,k,L):
    '''Logistic function'''
    return L/(1+np.exp(-k*(x-x0))) + y0

def get_fit(df,col,init_vals):
    '''Fit the logistic model and get the best parameters'''
    t = np.array(df.index)
    od = np.array(df[col],dtype=float)
    try:
        best_vals, covar = curve_fit(logistic, t, od, p0=init_vals)
        return best_vals
    except RuntimeError:
        return [0,0,0,0]

def getK(df,i,l):
    k = []
    for j in range(4):
        col = str(i+1)
        row = alph[l+j]
        fit = get_fit(df,(row+col),[13,0.3,0.7,0.8])
        k.append(fit[2])
    return k

In [4]:
K_msn4 = [getK(df_msn4,i,4) for i in range (len(pesticides))]
K_msn2 = [getK(df_msn2,i,4) for i in range (len(pesticides))]
K_yap1 = [getK(df_yap1,i,4) for i in range (len(pesticides))]
K_wt = [getK(df_wt,i,4) for i in range (len(pesticides))]

In [5]:
df_Ks = pd.DataFrame({
    'MSN4':K_msn4,
    'MSN2':K_msn2,
    'YAP1':K_yap1,
    'WT':K_wt,
})

In [12]:
vis_1 = 9*[True]+27*[False]
vis_2 = 9*[False] + 9*[True]+18*[False]
vis_3 = 18*[False] + 9*[True]+9*[False]
vis_4 = 27*[False]+9*[True]

In [13]:
fig = go.Figure()

# For each strain and pesticides, add a trace (the corresponding boxplot) to the figure. 
for column in df_Ks.columns.to_list():
    for yd,xd in zip(df_Ks[column],pesticides):
        fig.add_trace(go.Box(
                y = yd,
                name = xd,
                boxpoints='all',
                whiskerwidth=0.4,
                marker_size=4,
                line_width=1))
        
# Dropdown menu.
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'MSN4',
                  method = 'update',
                  args = [{'visible': vis_1}, # the index of True aligns with the indices of plot traces
                          {'title': 'MSN4',
                           'showlegend':True}]),
             dict(label = 'MSN2',
                  method = 'update',
                  args = [{'visible': vis_2},
                          {'title': 'MSN2',
                           'showlegend':True}]),
             dict(label = 'YAP1',
                  method = 'update',
                  args = [{'visible': vis_3},
                          {'title': 'YAP1',
                           'showlegend':True}]),
             dict(label = 'WT',
                  method = 'update',
                  args = [{'visible': vis_4},
                          {'title': 'WT',
                           'showlegend':True}]),
            ])
        )
    ])


<h4> Multiple tests with Benjamini/Hochberg correction </h4>

In [16]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [25]:
# Compute p-values for t-test on the values of K's for each strain.
p_vals_msn2 =[ttest_ind(elt,K_msn2[-1]).pvalue for elt in K_msn2[:-1]]
p_vals_msn4 =[ttest_ind(elt,K_msn4[-1]).pvalue for elt in K_msn4[:-1]]
p_vals_yap1 =[ttest_ind(elt,K_yap1[-1]).pvalue for elt in K_yap1[:-1]]
p_vals_wt =[ttest_ind(elt,K_wt[-1]).pvalue for elt in K_wt[:-1]]

In [46]:
# Multiple testing for the Benjamini-Hochberg procedure.
multipletests(p_vals_msn2,method='fdr_bh')

(array([False, False, False, False,  True,  True, False, False]),
 array([0.06020381, 0.06020381, 0.09068768, 0.40938372, 0.02881275,
        0.02881275, 0.45120165, 0.47114084]),
 0.006391150954545011,
 0.00625)

Multiple testing with Benjamini-Hochberg procedure (correcting for False Discovery Rate): only for MSN2 we get two significant tests for the pesticides Diazinon and Metolachlore.